In [1]:
from __future__ import print_function
from sklearn.utils.extmath import randomized_svd
import numpy as np

WordVecObject = open("../../Data/PretrainedWV/glove.42B.300d.txt", 'r', encoding='utf-8')

WordVecObject.readline()
Word2Idx = {}
WordVec_np = []
idx = 0
for line in WordVecObject:
    line = line.split()
    if len(line) != 301: continue
    word = line[0]
    vectors = list(map(float, line[1:]))
    Word2Idx[word] = idx
    WordVec_np.append(vectors)
    idx = idx+1
WordVec_np = np.array(WordVec_np)
Idx2Dict = dict((v,k) for k,v in Word2Idx.items())
print(len(WordVec_np))

1917493


In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook

BatchSize = 2**10
Threshold = 0.9
Dim = 300

U, Sigma, VT = randomized_svd(WordVec_np, n_components=Dim)
WtoW = np.matmul(U, np.diag(Sigma))

pbar = tqdm_notebook(total = WtoW.shape[0])
ExtractedWords = {}
for idx in range(int(WtoW.shape[0]/BatchSize)):
    start_idx = idx * BatchSize
    end_idx = start_idx + BatchSize
    if end_idx > WtoW.shape[0]: end_idx = WtoW.shape[0]
    pbar.update(end_idx - start_idx)

    CosSimMat = cosine_similarity(WtoW[start_idx:end_idx], WtoW)
    CosSimMat_idx = np.where(CosSimMat>=Threshold)
    
    for i in range(end_idx - start_idx):
        w1 = CosSimMat_idx[0][i]
        w2 = CosSimMat_idx[1][i]
        if w1 == w2-start_idx: continue
        else:
            try:
                if Idx2Dict[w1] in ExtractedWords:
                    ExtractedWords[Idx2Dict[w1]].append(Idx2Dict[w2])
                else:
                    ExtractedWords[Idx2Dict[w1]] = [Idx2Dict[w2]]
            except KeyError:
                print(w1, w2)
                continue
pbar.close()

In [3]:
FileObject = open("./Extracted_GloVe_Over"+str(Threshold)+"_Dim"+str(Dim)+"_Batch"+str(BatchSize)+".txt", 'w', encoding='utf-8')
print("./Extracted_GloVe_Over"+str(Threshold)+"_Dim"+str(Dim)+"_Batch"+str(BatchSize)+".txt")
for word, pairs in ExtractedWords.items():
    FileObject.write(word+' ')
    FileObject.write(' '.join(pairs)+'\n')
FileObject.close()

./Extracted_GloVe_Over0.9_Dim300_Batch1024.txt
